In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # ✅ Disable GPU (Force CPU)

In [2]:
# Import required libraries
import tensorflow as tf

import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib


In [3]:
# Load the dataset
# file_path = "Leak-detection-FrontEnd\pipeline_leak_detection_with_timestamp.csv"  # Change this if needed
df = pd.read_csv("pipeline_leak_detection_with_timestamp.csv")

# Convert Timestamp to datetime
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

# Sort by time (important for LSTM)
df = df.sort_values("Timestamp")

# Select Features & Target
X = df.drop(columns=["Leakage", "Timestamp"])  # Drop timestamp
y = df["Leakage"]

# Standardize the Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Save scaler for future use
joblib.dump(scaler, "scaler.pkl")


['scaler.pkl']

In [4]:
# Function to create time-series sequences for LSTM
def create_sequences(X, y, time_steps=10):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i + time_steps])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

# Define time step (how many previous data points to use for prediction)
time_steps = 10

# Reshape data
X_lstm, y_lstm = create_sequences(X_scaled, y, time_steps)

# Train-Test Split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X_lstm, y_lstm, test_size=0.2, random_state=42, stratify=y_lstm)

# Print shape of final data
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


X_train shape: (7992, 10, 3), y_train shape: (7992,)
X_test shape: (1998, 10, 3), y_test shape: (1998,)


In [5]:
# Build LSTM Model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(time_steps, X_train.shape[2])),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Binary Classification
])

# Compile Model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Print Model Summary
model.summary()


C:\Users\Vibhu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 10, 64)         │        17,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,857 (116.63 KB)

 Trainable params: 29,857 (116.63 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Train the Model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                    epochs=30, batch_size=64, verbose=1)


Epoch 1/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.8842 - loss: 0.4317 - val_accuracy: 0.9199 - val_loss: 0.2794
Epoch 2/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9212 - loss: 0.2807 - val_accuracy: 0.9199 - val_loss: 0.2812
Epoch 3/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9141 - loss: 0.2959 - val_accuracy: 0.9199 - val_loss: 0.2823
Epoch 4/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9209 - loss: 0.2797 - val_accuracy: 0.9199 - val_loss: 0.2793
Epoch 5/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9141 - loss: 0.2963 - val_accuracy: 0.9199 - val_loss: 0.2792
Epoch 6/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9220 - loss: 0.2758 - val_accuracy: 0.9199 - val_loss: 0.2793
Epoch 7/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9231 - loss: 0.2727 - val_accuracy: 0.9199 - val_loss: 0.2790
Epoch 8/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.9171 - loss: 0.2880 - val_accu

In [7]:
# Predict on Test Set
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Accuracy Score
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
Model Accuracy: 0.9199


In [8]:
model.save("lstm_pipeline_leak_model.h5")